In [30]:
# import libraries
import numpy as np
import pandas as pd
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

In [6]:
# read in data
datafile = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

print(datafile.head())

Y_train = datafile['species']
X_train = datafile.drop('species', axis=1)

# not looking at actual images for first pass, so drop id column from both test and train
X_train = X_train.drop('id', axis=1)
X_test = test.drop('id', axis=1)

# one hot encode species
lb = preprocessing.LabelBinarizer()
lb.fit(Y_train)

   id                species   margin1   margin2   margin3   margin4  \
0   1            Acer_Opalus  0.007812  0.023438  0.023438  0.003906   
1   2  Pterocarya_Stenoptera  0.005859  0.000000  0.031250  0.015625   
2   3   Quercus_Hartwissiana  0.005859  0.009766  0.019531  0.007812   
3   5        Tilia_Tomentosa  0.000000  0.003906  0.023438  0.005859   
4   6     Quercus_Variabilis  0.005859  0.003906  0.048828  0.009766   

    margin5   margin6   margin7  margin8    ...      texture55  texture56  \
0  0.011719  0.009766  0.027344      0.0    ...       0.007812   0.000000   
1  0.025391  0.001953  0.019531      0.0    ...       0.000977   0.000000   
2  0.003906  0.005859  0.068359      0.0    ...       0.154300   0.000000   
3  0.021484  0.019531  0.023438      0.0    ...       0.000000   0.000977   
4  0.013672  0.015625  0.005859      0.0    ...       0.096680   0.000000   

   texture57  texture58  texture59  texture60  texture61  texture62  \
0   0.002930   0.002930   0.03515

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [32]:
model = Sequential([
    Dense(2000, input_dim=X_train.shape[1]),
    Activation('relu'),
    Dropout(0.5),
    Dense(2000),
    Activation('relu'),
    Dropout(0.5),
    Dense(lb.classes_.size),
    Activation('softmax'),
])

model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(
    X_train.as_matrix(), 
    lb.transform(Y_train), 
    validation_split=0.2,
    nb_epoch=30
)

Train on 792 samples, validate on 198 samples
Epoch 1/30
792/792 [==============================] - 1s - loss: 4.5315 - acc: 0.0530 - val_loss: 4.4012 - val_acc: 0.0808
Epoch 2/30
792/792 [==============================] - 1s - loss: 3.8761 - acc: 0.1528 - val_loss: 3.6562 - val_acc: 0.1465
Epoch 3/30
792/792 [==============================] - 1s - loss: 2.9228 - acc: 0.2942 - val_loss: 2.8721 - val_acc: 0.1869
Epoch 4/30
792/792 [==============================] - 1s - loss: 2.2945 - acc: 0.3914 - val_loss: 2.1426 - val_acc: 0.4343
Epoch 5/30
792/792 [==============================] - 1s - loss: 1.7699 - acc: 0.5240 - val_loss: 1.7870 - val_acc: 0.4697
Epoch 6/30
792/792 [==============================] - 1s - loss: 1.4119 - acc: 0.6010 - val_loss: 1.3611 - val_acc: 0.6263
Epoch 7/30
792/792 [==============================] - 1s - loss: 1.1838 - acc: 0.6503 - val_loss: 1.1425 - val_acc: 0.6869
Epoch 8/30
792/792 [==============================] - 1s - loss: 0.9409 - acc: 0.7361 - val_l

In [34]:
predictions = model.predict_proba(X_test.as_matrix())

594/594 [==============================] - 0s     


In [35]:
labels = test['id']
headers = lb.classes_.tolist()
headers = ['id']+headers
results = pd.DataFrame(data=predictions)
labels = pd.DataFrame(labels)
results = pd.concat([labels, results], axis=1)
results.columns = headers

array([[  2.93434432e-09,   5.51064348e-13,   3.80511898e-12, ...,
          2.59748560e-08,   3.77012657e-11,   1.16432688e-12],
       [  1.69020790e-13,   2.16540244e-10,   3.55619922e-10, ...,
          1.08460590e-07,   8.19891775e-14,   2.25564605e-07],
       [  2.22839294e-08,   9.65896547e-01,   5.56013333e-07, ...,
          7.63894705e-16,   1.69151918e-15,   1.39521844e-05],
       ..., 
       [  3.87928048e-06,   1.15771219e-10,   1.05911502e-10, ...,
          1.66733713e-07,   1.03900536e-12,   8.56486281e-07],
       [  7.97952676e-12,   7.67831632e-12,   5.98419283e-04, ...,
          6.95353108e-10,   2.43126136e-12,   6.96372057e-11],
       [  2.63293923e-23,   5.91441479e-12,   3.50027896e-09, ...,
          3.19133001e-12,   8.75115054e-15,   3.22557160e-14]], dtype=float32)